In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [70]:
df = pd.read_csv('energydata_complete.csv')

In [71]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [72]:
df.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [73]:
# Celsius = T2, building = T6
simple_linear_reg_df = df[['T2', 'T6']].sample(15, random_state=2)

In [74]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['T2'], df['T6'],
test_size=0.3, random_state=1)

In [75]:
from sklearn.linear_model import LinearRegression

In [76]:
from sklearn.metrics import r2_score
r2_score = r2_score(df['T2'],df['T6'])
round(r2_score, 3)

-35.393

In [84]:
# Drop data, lights
df = df.drop(['date','lights'], axis = 1)

In [86]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

normalised_df = pd.DataFrame(scaler.fit_transform(df))
features_df = df
heating_target = df['Appliances']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_df, heating_target,
test_size=0.3, random_state=42)

In [87]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
predicted_values = linear_model.predict(x_test)

In [88]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 3) 

0.0

In [89]:
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 3)

0.0

In [90]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 2)

0.0

In [91]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 3) 

1.0

In [95]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(x_train, y_train)

def get_weights_df(model, feat, col_name):

    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')

In [96]:
linear_model_weights

,Features,Linear_Model_Weight
0,T1,-7.857404e-15
1,RH_3,-1.395654e-15
2,RH_1,-1.150490e-15
3,T9,-8.738686e-16
4,T8,-7.618924e-16
5,T_out,-6.953455e-16
6,RH_7,-5.088051e-16
7,RH_out,-6.369551e-17
8,Windspeed,-5.679202e-17
9,RH_5,-3.563970e-17


In [97]:
ridge_weights_df

,Features,Ridge_Weight
0,T9,-8.441097e-08
1,T2,-7.331831e-08
2,RH_2,-5.484818e-08
3,T_out,-4.419378e-08
4,RH_8,-2.307085e-08
5,RH_9,-7.037958e-09
6,RH_7,-6.758006e-09
7,T5,-6.397155e-09
8,RH_out,-4.363403e-09
9,T1,-1.484681e-09


In [98]:
lasso_weights_df

,Features,Lasso_weight
0,T7,0.0
1,Tdewpoint,0.0
2,Visibility,0.0
3,Windspeed,0.0
4,RH_out,-0.0
5,Press_mm_hg,-0.0
6,T_out,0.0
7,RH_9,-0.0
8,T9,0.0
9,RH_8,-0.0


In [99]:
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,T1,-7.857404e-15,-1.484681e-09,0.0
1,RH_3,-1.395654e-15,1.917548e-08,0.0
2,RH_1,-1.150490e-15,6.507041e-08,0.0
3,T9,-8.738686e-16,-8.441097e-08,0.0
4,T8,-7.618924e-16,3.988340e-08,0.0
5,T_out,-6.953455e-16,-4.419378e-08,0.0
6,RH_7,-5.088051e-16,-6.758006e-09,-0.0
7,RH_out,-6.369551e-17,-4.363403e-09,-0.0
8,Windspeed,-5.679202e-17,8.905214e-09,0.0
9,RH_5,-3.563970e-17,1.028186e-09,0.0
